# Data Collection

This notebook will focus on collecting ticket related data through Zendesk APIs.  The types of data that will try to be collected, but not an exhaustive list:
* Number of comments
* Total length of ticket (from open to close)
* First reply time
* Other metrics provided by the APIs

In looking through the API documentation and some of the responses in my testing, I think the following approach could get me what I need:

1. Use the [search API](https://developer.zendesk.com/api-reference/ticketing/ticket-management/search/) in order to find closed tickets that have a satisfaction rating (good, goodwithcomment, bad, badwithcomment)
    1. Will need to use pagination and other techniques because of the limitations of the search API
        1. specifically, it looks like the correct method is to use the [export search results](https://developer.zendesk.com/api-reference/ticketing/ticket-management/search/#export-search-results) approach
1. From the results of the previous step, use the [ticket metrics API](https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_metrics/) to get metric information
1. If sentinment analysis is also desired, would need to use [List Comments API](https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_comments/#list-comments) to get the comment data from the ticket IDs identified from previous steps

## Import Libraries

In [1]:
import pandas as pd
import requests
from time import sleep
from getpass import getpass
from urllib.parse import urlencode

## Create Functions for Data Collection

Here I will define some functions that will be needed to collect the ticket data.  From the outline above and translating to what is written below, you'll find:
    
* Collection functions that are interfacing with the various Zendesk APIs to:
    * Query for list of tickets based on criteria of searching for tickets with good or bad ratings
    * Retrieving metrics for the tickets identified in the search
    * Retrieving all comments for the tickets identified in the search
<p></p>

* Parsing functions to process response data from various API requests:
    * Ticket Search Responses
    * Ticket Metrics Responses
    * Ticket Comments
    
**Note: Zendesk API key and valid agent email address are required to access this data**

### Locate Tickets with Satisfaction Ratings

The following functions will utilize the Zendesk Search API endpoint to query based on ticket properties of having either a good or bad customer satisfaction rating.

[Search API](https://developer.zendesk.com/api-reference/ticketing/ticket-management/search/)

What I'll use below is the [Export Search Results](https://developer.zendesk.com/api-reference/ticketing/ticket-management/search/#export-search-results) endpoint of the Search API.

* `GET /api/v2/search/export?query={query}`

>Exports a set of results. See Query basics for the syntax of the query parameter.
>
>This endpoint is for search queries that will return more than 1000 results. The result set is ordered only by the >`created_at` attribute.
>The search only returns results of a single object type. The following object types are supported: ticket, organization, >user, or group.
>You must specify the type in the `filter[type]` parameter. Searches with type in the query string will result in an >error.
>
>Pagination
>* Cursor pagination
>
>Returns a maximum of 1000 records per page. The number of results shown in a page is determined by the `page[size]` >parameter.

In [2]:
def parse_ticket_search_results(results):
    '''
    Returns list of dictionaries with only certain predefined values 
    stripped out of the search results for tickets with satisfaction
    data. For use in conjunction with the collect_tickets_with_satisfaction
    function.
    
    Returns id, type, subject, description, status, group_id, channel,
    satisfaction score, and satisfaction comment.
    '''
    outputs = []
    fields = ['id', 
              'type', 
              'subject', 
              'description', 
              'status', 
              'group_id',
              'recipient'
             ]
    
    # also want 'via.channel', 'satisfaction_rating.score', 'satisfaction_rating.comment'
    # strip out wanted data from each result
    for result in results:
        # parse all the top level values identified in the fields list
        output = {key : value for key, value in result.items() if key in fields}
        
        # retrieve nested values for additional data
        output['channel'] = result['via'].get('channel')
        output['sat_score'] = result['satisfaction_rating'].get('score')
        output['sat_comment'] = result['satisfaction_rating'].get('comment')
        
        # append dictionary to the list that will be returned
        outputs.append(output)
    
    # return the list of dictionaries
    return outputs

In [3]:
def collect_tickets_with_satisfaction():
    '''
    Uses the Zendesk search API to find tickets that are closed and have
    satisfaction scores.  Returns a dataframe with columns:
        id, type, subject, description, status, group_id,
        channel, satisfaction score, satisfaction comment (if any)
        
    Depdency on the parse_ticket_search_results function.
    '''
    # initiative requests session
    email = input('Email for query: ')
    key = getpass('API key for query: ')
    credentials = (f'{email}/token', key)
    session = requests.Session()
    session.auth = credentials

    # define parameters for Zendesk search query
    params = {
        'query': 'type:ticket status:closed -satisfaction:offered -satisfaction:unoffered',
        'page[size]': 250,
        'filter[type]': 'ticket'
    }

    # create empty list to hold data
    search_results = []

    # construct the url based on the parameters
    base_url = 'https://[redacted].zendesk.com/api/v2/search/'

    # retrieve total number of records that match the query
    num_results_url = f'{base_url}count?{urlencode({key: item for key, item in params.items() if key == "query"})}'
    num_results = session.get(num_results_url)
    num_results = num_results.json()
    
    # output some information about number of tickets and rate of collection
    print(f'Collecting {num_results["count"]} tickets with satisfaction results\n\
    Collecting {params["page[size]"]} per page\n')
    
    # build url for request
    url = f'{base_url}export?{urlencode(params)}'

    while url:
        # output to let user know this is still running
        print(f'{len(search_results)}', end=' ')

        response = session.get(url)
        
        # catch timeout
        if response.status_code == 429:
            sleep(res.headers['retry-after'])
            
        # catch any other response other than success (200)
        elif response.status_code != 200:
            print('Status:', response.status_code, 'Problem with the request. Exiting.')
            exit()
        
        # if success
        else:
            data = response.json()
            
            # parse and add results to list
            results = pd.DataFrame(parse_ticket_search_results(data['results']))
            search_results.append(results)
            
            # cursor pagination
            if data['meta']['has_more']:
                url = data['links']['next']
            else:
                url = None

    # clean up variables and close requests session
    email = ''
    key = ''
    session.close()
    
    # return dataframe
    return pd.concat(search_results)

#### Collect and Export

In [ ]:
sat_tix = collect_tickets_with_satisfaction()

In [ ]:
sat_tix.head()

In [67]:
# export to CSV
sat_tix.to_csv('../data/sat_tix.csv', index=False)

### Get Ticket Metrics

See [Show Ticket Metrics](https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_metrics/#show-ticket-metrics)

`GET /api/v2/tickets/{ticket_id}/metrics`
* Returns a specific metric, or the metrics of a specific ticket.
* Returns a maximum of 100 records per page.
* Supports cursor (recommended) and offset pagination

returns [JSON Format](https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_metrics/#json-format)

It looks like the API returns a row for calendar measurements and a row for business hours measurements.
I think having both is too much data, so I'll write a function, like the satisfaction parser to parse the API return data and only return data for calendar, as for this project I think this metric will be the most germane as that is what the customer experiences (calendar time)

In [4]:
def parse_metrics(metrics):
    '''
    Returns list of dictionaries with only certain predefined values 
    stripped out of the ticket metric data. Returns all metrics, and 
    retrieves the 'calendar' time values for all the time-based minute 
    counts for the tickets.
    
    For use with the get_ticket_metrics function.
    
    https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_metrics/#json-format
    '''
    outputs = []
    nested = ['reply_time_in_minutes', 
              'first_resolution_time_in_minutes',
              'full_resolution_time_in_minutes', 
              'agent_wait_time_in_minutes', 
              'requester_wait_time_in_minutes', 
              'on_hold_time_in_minutes'
             ]
    
    # parse all the top level values identified in the fields list
    output = {key : value for key, value in metrics.items() if key not in nested}

    # retrieve nested values for additional data
    for key in nested:
        output[key] = metrics[key].get('calendar')

    # append dictionary to the list that will be returned
    outputs.append(output)

    # return the list of dictionaries
    return outputs

In [5]:
def get_ticket_metrics(ids):
    '''
    Takes a list of ticket ids and returns a dataframe with ticket metric
    information for the given ticket ids.  
    
    Dependency on the parse_metrics function.
    
    For information returned, see:
    https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_metrics/#json-format
    '''
     # initiative requests session
    email = input('Email for query: ')
    key = getpass('API key for query: ')
    credentials = (f'{email}/token', key)
    session = requests.Session()
    session.auth = credentials

    # create empty list to hold data
    metrics_results = []

    # construct the url based on the parameters
    base_url = 'https://[redacted].zendesk.com/api/v2/tickets/'

    # output some information about number of tickets and rate of collection
    print(f'Collecting metrics for the {len(ids)} ticket IDs passed\n')
    
    # will need to be careful for rate limits on this since we need to collect
    # metrics from one ticket at a time.  Rate limit is 100/min
    
    for id_ in ids:
        
        # build url for request
        url = f'{base_url}{id_}/metrics'

        # output to let user know this is still running
        print('.', end='')

        response = session.get(url)

        # catch timeout
        if response.status_code == 429:
            sleep(res.headers['retry-after'])
            response = session.get(url)

        # catch any other response other than success (200)
        elif response.status_code != 200:
            print('Status:', response.status_code, 'Problem with the request. Exiting.')
            exit()

        # if success
        data = response.json()

        # parse and add results to list
        results = pd.DataFrame(parse_metrics(data['ticket_metric']))
        metrics_results.append(results)
        sleep(1)

    # clean up variables and close requests session
    email = ''
    key = ''
    session.close()
    
    # return dataframe
    return pd.concat(metrics_results)

#### Collect and Export

In [ ]:
# retrieve ticket metric for all the tickets collected from the search for
# tickets with good or bad satisfaction ratings
tix_mets = get_ticket_metrics(list(sat_tix['id']))

In [93]:
tix_mets.shape

(13323, 22)

In [94]:
# export to CSV
tix_mets.to_csv('../data/tix_mets.csv', index=False)

### Get Ticket Comment Data

To retrieve all the comments for a given ticket ID, I the [List Comments API](https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_comments/#list-comments) appears to the the correct one.

List Comments
* `GET /api/v2/tickets/{ticket_id}/comments`

Returns the comments added to the ticket.

Each comment may include a content_url for an attachment or a recording_url for a voice comment that points to a file that may be hosted externally. For security reasons, take care not to inadvertently send Zendesk authentication credentials to third parties when attempting to access these files. See Working with url properties.

Pagination:
* Cursor pagination (recommended)
* Offset pagination

Returns a maximum of 100 records per page.

In [30]:
def parse_comments(comments, ticket_id):
    '''
    Returns list of dictionaries with only certain predefined values 
    stripped out of the ticket comment data.
    
    Returns tbd.
    '''
    outputs = []
    exclude = ['via', 
              'metadata',
               'attachments'
             ]
    
    for comment in comments:
        # parse all the top level values identified in the fields list
        output = {key : value for key, value in comment.items() if key not in exclude}

        # retrieve nested value of interest
        output['channel'] = comment['via'].get('channel')
        
        # add ticket_id for relation to original ticket
        output['ticket_id'] = ticket_id

        # append dictionary to the list that will be returned
        outputs.append(output)

    # return the list of dictionaries
    return outputs

In [36]:
def collect_tickets_comments(ids):
    '''
    Uses the Zendesk ticket comments API to retrieve comment data from a list 
    of ticket ids provided.  This will return a dataframe of ticket comment
    data.
    
    https://developer.zendesk.com/api-reference/ticketing/tickets/ticket_comments/
    '''
    # initiative requests session
    email = input('Email for query: ')
    key = getpass('API key for query: ')
    credentials = (f'{email}/token', key)
    session = requests.Session()
    session.auth = credentials

    # define parameters for Zendesk search query
    params = {
        'sort_order': 'asc',
        'include_inline_images': 'false'
    }

    # create empty list to hold data
    comments = []

    # construct the url based on the parameters
    base_url = 'https://[redacted].zendesk.com/api/v2/tickets/'
    
    # output some information about number of tickets and rate of collection
    print(f'Collecting comments from {len(ids)} ticket IDs passed\n')
    
    for id_ in ids:
        # build url for request
        url = f'{base_url}{id_}/comments?{urlencode(params)}'
        
        # cursor pagination
        while url:
            # output to let user know this is still running
            print(f'{len(comments)}', end=' ')

            response = session.get(url)

            # catch timeout
            if response.status_code == 429:
                sleep(res.headers['retry-after'])

            # catch any other response other than success (200)
            elif response.status_code != 200:
                print('Status:', response.status_code, 'Problem with the request. Exiting.')
                exit()

            # if success
            else:
                data = response.json()

                # parse and add results to list
                results = pd.DataFrame(parse_comments(data['comments'], id_))
                comments.append(results)
                # print(type(data['comments']))
                # cursor pagination
                if data['next_page']:
                    url = data['next_page']
                else:
                    url = None

        # sleep as to not upset the rate limit
        sleep(1)
        
    # clean up variables and close requests session
    email = ''
    key = ''
    session.close()
    
    # return dataframe
    return pd.concat(comments)

#### Collect and Export

In [ ]:
# collect ticket comments for ticket ids from search
tix_comments = collect_tickets_comments(list(sat_tix['id']))

In [38]:
tix_comments.shape

(85692, 11)

In [39]:
tix_comments.to_csv('../data/tix_comments.csv', index=False)

In [ ]:
tix_comments.head()